# Source images in directory and its subdirectories

In [ ]:
import os
import shutil
import pandas as pd
import re

def clean_string(s):
    # Remove non-alphanumeric characters and convert to lowercase
    return re.sub(r'\W+', '', s).lower()

def find_and_copy_images(source_dir, destination_dir, show_names, folder_type):
    # Walk through all subdirectories to find specified folder type (Poster or Still)
    for root, dirs, files in os.walk(source_dir):
        for dir_name in dirs:
            if dir_name.lower() == folder_type.lower():
                target_folder = os.path.join(root, dir_name)
                print(f'Found "{folder_type}" folder: {target_folder}')
                
                # Walk through the specified folder to find images
                for sub_root, sub_dirs, sub_files in os.walk(target_folder):
                    for file in sub_files:
                        for show in show_names:
                            if clean_string(show) in clean_string(file):
                                # Construct the full file path
                                file_path = os.path.join(sub_root, file)
                                # Copy the file to the destination directory
                                shutil.copy(file_path, destination_dir)
                                print(f'Copied {file_path} to {destination_dir}')
                                break  # Avoid copying the same file multiple times if it matches multiple shows

# Function to read show names from the first column of a given sheet
def get_show_names(excel_file_path, sheet_name):
    df = pd.read_excel(excel_file_path, sheet_name=sheet_name, usecols=[0])
    if df.empty:
        print(f"No data found in the first column of the {sheet_name} sheet.")
        return []
    return df.iloc[:, 0].dropna().tolist()

# Prompt the user for paths to define destination directories for TV and movie posters and stills
print("Please enter the paths for the destination directories:")

movie_poster_destination_directory = input("Enter the path for Movie Poster destination (Press Enter): ").strip()
movie_still_destination_directory = input("Enter the path for Movie Still destination (Press Enter): ").strip()

tv_poster_destination_directory = input("Enter the path for TV Poster destination (Press Enter): ").strip()
tv_still_destination_directory = input("Enter the path for TV Still destination (Press Enter): ").strip()

# Ensure the destination directories exist
os.makedirs(movie_poster_destination_directory, exist_ok=True)
os.makedirs(movie_still_destination_directory, exist_ok=True)
os.makedirs(tv_poster_destination_directory, exist_ok=True)
os.makedirs(tv_still_destination_directory, exist_ok=True)

# Prompt the user for the Excel file path to get the list of TV show and movie titles
excel_file_path = input("Enter the path to the Excel file with show titles (Press Enter): ").strip()

# Extract movie titles from the first column of the "Movies" tab
movie_show_names = get_show_names(excel_file_path, 'Movies')

# Extract TV show titles from the first column of the "TV" tab
tv_show_names = get_show_names(excel_file_path, 'TV')

# Define source directories for TV and movies (these paths remain static in this example)
movie_source_directory = r'Z:\3_ContentDepartment\2. Content research\Image Library\Movies'
tv_source_directory = r'Z:\3_ContentDepartment\2. Content research\Image Library\TV'

# Call the function for Movie Poster images
find_and_copy_images(movie_source_directory, movie_poster_destination_directory, movie_show_names, 'Poster')

# Call the function for Movie Still images
find_and_copy_images(movie_source_directory, movie_still_destination_directory, movie_show_names, 'Still')

# Call the function for TV Poster images
find_and_copy_images(tv_source_directory, tv_poster_destination_directory, tv_show_names, 'Poster')

# Call the function for TV Still images
find_and_copy_images(tv_source_directory, tv_still_destination_directory, tv_show_names, 'Still')

print("Image search and copy process completed.")


Please enter the paths for the destination directories:


# Resize Multiple dimensions (Posters)

In [ ]:
import os
from PIL import Image

# Instructions for the user
print("Welcome to the Image Resizing and Cropping Script!")
print("Please fill in the following information before running the script:")

# Prompt for source directory path
source_dir = input("Enter the source directory path: ").strip()

# Prompt for base destination directory path
base_destination_dir = input("Enter the base destination directory path: ").strip()

# List of dimensions to resize to
print("\nSelect the Size required (enter dimensions as width,height): eg.:600,900")
dimensions = []
while True:
    dimension_input = input("Enter dimension (or 'done' to finish): ").strip()
    if dimension_input.lower() == 'done':
        break
    try:
        width, height = map(int, dimension_input.split(','))
        dimensions.append((width, height))
    except ValueError:
        print("Invalid input. Please enter dimensions as width,height.")

def resize_and_crop(img, size):
    """
    Resize and crop an image to fit the specified size without stretching.
    
    :param img: PIL Image object
    :param size: tuple of (width, height)
    """
    img_ratio = img.width / float(img.height)
    target_ratio = size[0] / float(size[1])

    if target_ratio > img_ratio:
        # The image is taller relative to the desired aspect ratio
        img = img.resize((size[0], int(size[0] / img_ratio)), Image.ANTIALIAS)
    else:
        # The image is wider relative to the desired aspect ratio
        img = img.resize((int(size[1] * img_ratio), size[1]), Image.ANTIALIAS)

    # Now crop to the exact size, centralized at the center
    width, height = img.size
    dx = (width - size[0]) // 2
    dy = (height - size[1]) // 2
    img = img.crop((dx, dy, dx + size[0], dy + size[1]))

    return img

def process_images(source_dir, base_destination_dir, dimensions):
    """
    Resize and crop images in the source directory and save them to the destination directories.
    
    :param source_dir: Source directory containing original images.
    :param base_destination_dir: Base destination directory where resized images will be saved.
    :param dimensions: List of tuples specifying the dimensions (width, height) for resizing.
    """
    # Ensure the base destination directory exists
    os.makedirs(base_destination_dir, exist_ok=True)
    
    # Walk through all files in the source directory
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                file_path = os.path.join(root, file)
                
                try:
                    # Open the image file
                    with Image.open(file_path) as img:
                        # Loop through each dimension and resize/crop the image
                        for width, height in dimensions:
                            img_resized = resize_and_crop(img, (width, height))
                            
                            # Create the output directory for the current dimension
                            dimension_folder = f'{width}x{height}'
                            destination_dir = os.path.join(base_destination_dir, dimension_folder)
                            os.makedirs(destination_dir, exist_ok=True)
                            
                            # Create the filename with dimensions appended
                            base_name, file_extension = os.path.splitext(file)
                            resized_filename = f"{base_name}-{width}x{height}.jpg"
                            resized_file_path = os.path.join(destination_dir, resized_filename)
                            
                            # Save the resized and cropped image as JPG
                            img_resized.save(resized_file_path, 'JPEG', quality=95)
                            print(f'Resized and cropped {file_path} to {resized_file_path}')
                
                except Exception as e:
                    print(f'Error processing file {file_path}: {e}')


        
        
# Call the function to resize and crop images with the provided parameters
process_images(source_dir, base_destination_dir, dimensions)

print("Image resizing and cropping process completed.")


# Resize Multiple dimensions (Stills)